In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
chat_model = ChatOpenAI()
llm = OpenAI()

In [ ]:
print(chat_model.model_name)

In [ ]:
# chat_model = ChatOpenAI(model = 'gpt-3.5-turbo-0125')
# print(chat_model.model_name)

In [ ]:
from langchain_core.messages import HumanMessage
text = "我想创办一家科技公司，关于人工智能的,帮我起个名字?"
messages = [HumanMessage(content=text)]

# llm.invoke(text)

chat_model.invoke(messages)

-----------------

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate


template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")


---------------

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [ ]:
joke_query = "Tell me a joke."

parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# print(parser.get_format_instructions())

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

----------

In [74]:
from langchain_core.messages import SystemMessage
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="人工智能能帮助老人做什么"),
]

chat_model.invoke(messages)


AIMessage(content='人工智能可以帮助老人做很多事情，例如：\n\n1. 提供医疗健康监测：智能健康监测设备可以监测老人的生理指标并提供实时反馈，有助于提前预防疾病。\n\n2. 语音助手和智能家居设备：智能语音助手（如Alexa、Google Assistant等）和智能家居设备可以帮助老人控制家电、查询信息、提醒日程等，提高生活便利性。\n\n3. 社交和娱乐：智能社交平台和娱乐应用可以帮助老人保持社交联系和娱乐活动，缓解孤独和抑郁。\n\n4. 记忆训练和认知锻炼：一些智能应用和游戏可以帮助老人进行记忆训练和认知锻炼，有助于保持大脑活力。\n\n总的来说，人工智能技术可以为老人提供更好的生活质量和健康管理服务。', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [ ]:
for chunk in chat_model.stream(messages):
    print(chunk.content, end="", flush=True)

--------------

In [68]:
import json
from langchain_core.utils.function_calling import convert_to_openai_tool

def multiply(a: int, b: int) -> int:
    """Multiply two integers together.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

print(json.dumps(convert_to_openai_tool(multiply), indent=2))



{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "type": "integer",
          "description": "First integer"
        },
        "b": {
          "type": "integer",
          "description": "Second integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


In [69]:
llm_with_tools = chat_model.bind_tools([multiply])
llm_with_tools.invoke("what is 3 * 12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mSijMNGnSYfXrqHUMW6nwHsq', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None})

----------

In [75]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# set_llm_cache(InMemoryCache())
set_llm_cache(None)

llm.predict("写一首四言绝句")

CPU times: user 8.97 ms, sys: 1.89 ms, total: 10.9 ms
Wall time: 1.4 s


'\n\n秋色落尽愁难散\n寒风吹过泪痕干\n枯枝凋零伤心处\n寂寞无言孤独寒'

In [76]:
%%time
# The second time it is, so it goes faster
llm.predict("写一首四言绝句")

CPU times: user 9.26 ms, sys: 1.82 ms, total: 11.1 ms
Wall time: 712 ms


'\n\n春风吹绿柳，花开笑脸红。\n鸟语声声绕，春梦悠悠飞。'

In [72]:
%%time
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

CPU times: user 7.64 ms, sys: 1.78 ms, total: 9.42 ms
Wall time: 1.18 s


"\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything."

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY=<your-api-key>